Imports

In [ ]:
!pip install av
!pip install scenedetect
!pip install ffmpeg-python
!pip install --upgrade pip
!pip install -q -U google-generativeai
!pip install --upgrade transformers datasets[audio] accelerate

In [ ]:
import os
import cv2
import time
import torch
import random
import ffmpeg
import warnings
import numpy as np
import pandas as pd
from PIL import Image
from glob import glob
import soundfile as sf
from json import loads,dumps
import matplotlib.pyplot as plt
from scipy.signal import resample
import typing_extensions as typing
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from scenedetect import open_video, VideoStreamCv2, SceneManager
from scenedetect.detectors import ContentDetector

In [ ]:
# Extracting all required ids 

df = pd.read_csv('/kaggle/input/youtube-data/ground_labels_new.csv') 

df['Primary Label'] = df['Primary Label'].str.lower() 
df = df[df['Primary Label'].isin(['appropriate', 'inappropriate'])]

ids_df = pd.read_csv('/kaggle/input/filtered-ground-truth/filtered_ground_truth.csv') 
ids_df['Video Id'] = ids_df['Video Id'].str.lstrip("'") 
ids_ = ids_df['Video Id'].to_list() 
df = df[df['Video Id'].isin(ids_)] 

transcriptions_df = pd.read_csv('/kaggle/input/youtube-data/eng-complete-transcriptions.csv')
df = pd.merge(df, transcriptions_df, on='Video Id', how='inner') 

df

In [ ]:
# Extracting video ids and primary labels

video_ids = list(df['Video Id'])
primary_labels = list(df['Primary Label'])
all_transcriptions = list(df['Transcription'])

In [ ]:
# Extracting data from transcripts

transcriptions = []
lengths = []

for (i, id_) in enumerate(video_ids):
    transcriptions.append(all_transcriptions[i].split("chunks")[0])
    lengths.append(len(all_transcriptions[i].split("chunks")[0]))


In [ ]:
import os 

available_ids = os.listdir('/kaggle/input/youtube-data/Ads/Ads') 
len(available_ids) 

Extracting Images 

In [ ]:
def detect_scenes(video_path, threshold = 30):
    """Detect scenes in a video and return scene start and end frames."""
    scene_list = []
    while len(scene_list) < 6 and threshold > 0:
        threshold //= 2
    
        video = open_video(video_path)
        scene_manager = SceneManager()
        scene_manager.add_detector(ContentDetector(threshold=threshold))
    
        scene_manager.detect_scenes(video)
        scene_list = scene_manager.get_scene_list()
    
    return scene_list


def get_top_n_longest_scenes(scene_list, n):
    '''Return the top n longest scenes with start and end frame indices.'''
    scene_durations = [(start, end - start) for start, end in scene_list]
    scene_durations.sort(key=lambda x: x[1], reverse=True)

    # Top n longest scenes with start and end frame indices
    longest_scenes = [(start, start + duration) for start, duration in scene_durations[:n]]
    return longest_scenes


def sort_scenes_by_frame(scenes_list):
    '''Sort scenes by their start frame number.'''
    sorted_scenes = sorted(scenes_list, key=lambda scene: scene[0].get_frames())
    return sorted_scenes


def get_num_grids(video_path):
    '''Get number of grids to be created'''
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = total_frames / fps

    # Calculate number of grids based on the duration
    duration = round(duration, 2)
    if ((duration // 60) + 1) <= 5:
        return int(((duration // 60) + 1))
    else:
        return 5
        

def extract_k_frames_from_scene(video_path, scene, k):
    '''Extract k frames evenly spaced from each scene.'''
    # Extract frame numbers from scene start and end
    start_frame = scene[0].get_frames() + 1
    end_frame = scene[1].get_frames() - 1

    # Create k equally spaced frame indices within the scene's range
    frame_indices = np.linspace(start_frame, end_frame, k, dtype=int)
    
    cap = cv2.VideoCapture(video_path)
    frames = []

    # Extract frames from calculated indices
    for frame_no in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    
    cap.release()
    return frames


def create_image_grid(frames, grid_size=(1000, 1000)):
    '''Arrange 6 frames into a 3x2 grid and resize to the specified grid size.'''
    # Ensure all frames have the same size for concatenation
    frames = [cv2.resize(frame, (640, 360)) for frame in frames]  # Resize to a common size like 640x360
    rows = [np.concatenate(frames[i:i+2], axis=1) for i in range(0, 6, 2)]
    image_grid = np.concatenate(rows, axis=0)
    
    return np.array(Image.fromarray(image_grid).resize(grid_size))

In [ ]:
def get_images(video_path, n=6):
    ''' 1. Detect scenes
        2. Get k; where k = num_grids
        3. Get the 6k longest scenes
        4. Sort scenes wrt frame numbers
        5. Extract 1 frame per 6k scene
        6. Create k image grids of 6 frames each
     '''
    scene_list = detect_scenes(video_path)
    k = get_num_grids(video_path)
    longest_scenes = get_top_n_longest_scenes(scene_list, n*k)
    scenes = sort_scenes_by_frame(longest_scenes)

    frames = []
    for scene in scenes:
        frames.extend(extract_k_frames_from_scene(video_path, scene, 1))

    grids = []
    for i in range(k):
        start_idx = i * n
        end_idx = start_idx + n
        grid_frames = frames[start_idx:end_idx]
        grid = create_image_grid(grid_frames, grid_size=(1000, 1000))
        grids.append(grid)

    return grids

In [ ]:
def get_images_2(video_path, n=6):
    ''' 
    Extracts image grids from video based on scenes.
    Falls back to uniform sampling if no scenes are detected.
    '''
    # Step 1: Detect scenes
    scene_list = detect_scenes(video_path)

    # Step 2: Get number of grids and total frames needed
    k = get_num_grids(video_path)
    total_frames_needed = n * k

    def extract_nk_frames(video_path, total_frames_needed):
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
        if total_frames == 0:
            cap.release()
            return []
    
        # Get `total_frames_needed` evenly spaced frame indices
        selected_indices = np.linspace(0, total_frames - 1, total_frames_needed, dtype=int)
    
        frames = []
        for idx in selected_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            success, frame = cap.read()
            if success:
                frames.append(frame)
    
            if len(frames) >= total_frames_needed:
                break
    
        cap.release()
        return frames

    # Case 1: No scenes detected → use full video sampling
    if not scene_list:
        frames = extract_nk_frames(video_path, total_frames_needed)

    # Case 2: Scenes detected → extract from each scene
    else:
        available_scenes = len(scene_list)
        frames_per_scene = total_frames_needed // available_scenes
        remaining_frames = total_frames_needed % available_scenes

        if available_scenes == 1:
            frames_per_scene = total_frames_needed
            remaining_frames = 0

        frames = []
        for i, scene in enumerate(scene_list):
            num_frames = frames_per_scene + (1 if i < remaining_frames else 0)
            frames.extend(extract_k_frames_from_scene(video_path, scene, num_frames))

        frames = frames[:total_frames_needed]

    # Step 3: Ensure enough frames for all grids
    if len(frames) < n:
        frames = frames * (n // len(frames)) + frames[:(n % len(frames))]

    # Step 4: Create image grids
    grids = []
    for i in range(k):
        start_idx = i * n
        end_idx = start_idx + n
        grid_frames = frames[start_idx:end_idx]
        if grid_frames:
            grid = create_image_grid(grid_frames, grid_size=(1000, 1000))
            grids.append(grid)

    return grids

In [ ]:
len(video_ids) 

In [ ]:
import google.generativeai as genai 

os.environ["API_KEY"] = ""

genai.configure(api_key=os.environ["API_KEY"]) 
model = genai.GenerativeModel("gemini-2.5-flash", system_instruction="You are an expert content moderator.") 

In [ ]:
import torch
print(torch.cuda.is_available())

Define Model 

In [ ]:
class LLM_Output(typing.TypedDict):
    label: str
    language: list[str]
    response: str

Prompt 

In [ ]:
prompt =  """"A video can be considered inappropriate for children if it contains physical violence (cartoonish, realistic, or gory), interpersonal violence (bullying, pranks, meanness, belittling, controlling behavior, talking down to others, or manipulation), self-harm or suicide (depictions of harm inflicted on oneself or suicidal thoughts/tendencies), extreme stunts (life-endangering, high-risk activities/challenges that require adult supervision), dangerous products or services (like paintball, airsoft, fireworks, weapons, hunting equipment, graffiti products), scary content (horror, suspense, loud violence, zombies, skeletons, masks, scary clowns, blood, dangerous fire, car crashes, medical procedures or other scary visuals), sexual content (innuendos, sexual behavior, nudity, suggestiveness), intimacy (romantic conduct, couples expressing love, sensual contact, dating, relationships or any expression of love or loss for a significant other,  lyrics about or mentions of love or romance, heartbreak), inappropriate clothing (significant skin exposure or sheer clothing outside of appropriate contexts such as sunbathing, swinmwear,etc), substance use (mention, depictions, or use of alcohol, drugs, tobacco), gambling (betting, gambling, casinos, online or real world casino games, online trading), offensive language (from mild to severe with words such as “stupid” or “idiot”), stereotypes (racial, ethnic, gender, body shaming), crude humor (inappropriate humor, innuendos), weapons (guns, swords, knives, etc but toy guns such as nerf guns are fine), unsuitable media (ads for violent or adult-themed video games, casino games, games with romance or battle or first person-shooter games), fight sports (boxing, wrestling, martial arts), financial content (difference, rolling spot forex, financial spread betting and synonymous products but banking ads are fine), health and beauty services (cosmetic procedures, weight loss, tanning, piercings, tattoos, cosmetics and makeup (no matter their essential benefits) and other products related to external personal care focused on body image but ads on fashion products and essential hygiene products like sunscreen, shampoo, toothpaste, etc. are fine, however, products like sanitary care and hair removal is inappropriate), ads or sponsored content for consumable food or drink products (regardless of nutritional content), where the product is consumed by a person or shown with branding, packaging, or promotional intent are inappropriate; general content like cooking shows, recipes, or scenes where food is simply present or being eaten without promotion are acceptable, pet/animal food products and healthcare, healthcare or medical topics (reproductive health, mental health, substance abuse, eating disorders, pharmaceuticals, health insurance, any nutritional or dietary supplements), sensitive topics (politics, religion, controversial social issues, astrology, occult, paranormal, religious sermons), privacy or safety risks (services or competitions requiring personal information, social networks for strangers, spy services, or video game skin trading), or any explicit depictions or discussions of death or dying. Consider that you are an expert content moderator. Based on the image grids provided containing frames from key scenes from within the video and its audio, is the video advertisement inappropriate for children on this definition? If it is, label it as “inappropriate.” If it is not, label it as “appropriate.” Justify your conclusion. Also, mention what language is being spoken if there is any. 
Make the outputs in JSON format: {“label”: “appropriate” or “inappropriate”, “justification”: “brief explanation of the classification”, “language”: “language detected in the video”}."""

prompt 

Define function

In [ ]:
len(video_ids)

In [ ]:
start = 0
end = len(video_ids)

In [ ]:
ids = []
predicted_labels = []
languages = []
responses = []
ground_truths = []
remaining = []
key_count = 0

img_dir = '/kaggle/working/Images'
if not os.path.exists(img_dir):
    os.makedirs(img_dir)
    
for i in range(start,end): 
    print('ID: ', video_ids[i])

    if video_ids[i] in available_ids:

        try:
            contents_of_ad = os.listdir('/kaggle/input/youtube-data/Ads/Ads/' + video_ids[i]) 
            contents_of_ad.remove('audio.mp3') 
            video_path = '/kaggle/input/youtube-data/Ads/Ads/' + video_ids[i] + '/' + contents_of_ad[0] 
            audio_path = f'/kaggle/input/youtube-data/Ads/Ads/{video_ids[i]}/audio.mp3'
            try:
                audio_file = genai.upload_file(path=audio_path, resumable=False)
            except Exception as e:
                print(f"Error uploading audio: {e}")
                remaining.append(video_ids[i])
                continue
            # Extract multiple images representative of the video
            try:
                images = get_images(video_path)
            except:
                images = get_images_2(video_path)
                
            # Save each image returned by extract_images_of_frames
            image_paths = []
            for idx, img in enumerate(images):
                # Convert NumPy array to PIL image
                image = Image.fromarray(img)
                
                # Save the image to a file
                image_name = f"{video_ids[i]}_{idx + 1}.png"
                image_path = os.path.join(img_dir, image_name)
                image.save(image_path)
                image_paths.append(image_path)
                print(image_path)
                
            # Display the images
            fig, axes = plt.subplots(1, len(images), figsize=(10, 3))
            if len(images) == 1:
                axes.imshow(images[0])
                axes.axis('off')
            else:
                for ax, img in zip(axes, images):
                    ax.imshow(img)
                    ax.axis('off')

            plt.tight_layout()
            plt.show()

            # Upload images and handle potential errors
            uploaded_files = []
            try:
                for image_path in image_paths:
                    uploaded_file = genai.upload_file(path=image_path, resumable=False)
                    uploaded_files.append(uploaded_file)
            except Exception as e:
                print(f"Error uploading images: {e}")
                remaining.append(video_ids[i])
                continue
                
            # Check if all images have uploaded
            try:
                for uploaded_file in uploaded_files:
                    while uploaded_file.state.name == "PROCESSING":
                        print('.', end='')
                        time.sleep(10)
                        uploaded_file = genai.get_file(uploaded_file.name)
                    if uploaded_file.state.name == "FAILED":
                        raise ValueError(uploaded_file.state.name)
            except Exception as e:
                print(f"Error during image processing: {e}")
                remaining.append(video_ids[i])
                continue

            # Make inference
            try:
                inputs_ = [audio_file] 
                inputs_.extend(uploaded_files) 
                inputs_.extend([prompt]) 
                response = model.generate_content(inputs_,
                                                  generation_config=genai.GenerationConfig(
                                                      response_mime_type="application/json",
                                                      temperature=0,  # Set temperature here 
                                                      response_schema=LLM_Output),
                                                  safety_settings={
                                                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                                                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                                                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                                                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
                                                  })
            except Exception as e:
                print(f"Error making inference: {e}")
                remaining.append(video_ids[i])
                continue

            # Wrap response.text access in try-except
            try:
                print("Completed for video number:", i, "\t", video_ids[i])

                dictionary = loads(response.text)
                print(dictionary)

                ids.append(video_ids[i])
                predicted_labels.append(dictionary['label'])
                languages.append(dictionary['language'])
                responses.append(dictionary['response'])
                ground_truths.append(primary_labels[i]) 
                
            except Exception as e:
                print(f"Error processing response.text: {e}")
                remaining.append(video_ids[i])
                continue

        except Exception as e:
            print(f"Unexpected error: {e}")
            remaining.append(video_ids[i])
            continue

        # Wait a bit to avoid exceeding rate limits
        time.sleep(30)

# At the end, print remaining videos
print("Remaining videos with errors:", remaining)

In [ ]:
results_dir = '/kaggle/working/results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [ ]:
# At the end, print remaining videos 
remaining_df = pd.DataFrame({'Video Id': remaining})

# Save to CSV
remaining_df.to_csv(f'/kaggle/working/results/DAVSP-gemini-audio-remaining-{start}-{end}.csv', index=False)

print("Remaining videos saved to remaining.csv")
print("Remaining videos with errors:", remaining) 

In [ ]:
new_df = pd.DataFrame({
    'Video Id': ids,
    'Primary Label': ground_truths,
    'Predicted Label': predicted_labels,
    'Response': responses, 
    'Languages': languages 
})

new_df.head() 

In [ ]:
new_df.to_csv(f'/kaggle/working/results/davsp_gemini-audio-{start}-{end}.csv', index=False)

In [ ]:
# Changing to binary lists 

predictions = [1 if pred == 'inappropriate' else 0 for pred in predicted_labels] 
ground_truths = [1 if label == 'inappropriate' else 0 for label in ground_truths] 

In [ ]:
# Obtaining classification report 
from sklearn.metrics import classification_report 

report = classification_report(ground_truths, predictions) 
print(report) 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ground_truths, predictions)

plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Appropriate', 'Inapproriate'], yticklabels=['Appropriate', 'Inapproriate'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()